In [2]:
pip install streamlit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 92.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [3]:
pip install langchain


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.2/290.2 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00


In [4]:
pip install openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 21.4 MB/s eta 0:00:00


In [5]:
pip install wikipedia-api


  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.7.1-py3-none-any.whl size=14347 sha256=9cc5c227c6ee40522a691f4ddb855ea2ccedb50d9d225dbc393deb175b23bf07
  Stored in directory: /root/.cache/pip/wheels/4c/96/18/b9201cc3e8b47b02b510460210cfd832ccf10c0c4dd0522962
Successfully built wikipedia-api


In [14]:
pip install langchain_community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.9 MB/s eta 0:00:00


In [17]:
pip install wikipedia-api


In [ ]:
import os
import streamlit as st
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
import wikipediaapi

# Set the OpenAI API key
os.environ['OPENAI_API_KEY'] = '"enter your secret key here'

# Streamlit App framework
st.title('🦜🔗 YouTube Script Assistant')  # Set the title
st.image('./Youtube.png')  # URL of YouTube logo
prompt = st.text_input('Plug in your prompt here')  # The box for the text prompt

# Prompt templates
title_template = PromptTemplate(
    input_variables=['topic'],
    template='Write me a YouTube video title about {topic}'
)

script_template = PromptTemplate(
    input_variables=['title', 'wikipedia_research'],
    template='Write me a YouTube video script based on this title: TITLE: {title} while leveraging this Wikipedia research: {wikipedia_research}'
)

# Memory for the title and script
title_memory = ConversationBufferMemory(input_key='topic', memory_key='chat_history')
script_memory = ConversationBufferMemory(input_key='title', memory_key='chat_history')

# Initialize the LLM
llm = OpenAI(temperature=0.9)  # You can adjust the temperature for creativity or focus

# Chains for title and script generation
title_chain = LLMChain(llm=llm, prompt=title_template, verbose=True, output_key='title', memory=title_memory)
script_chain = LLMChain(llm=llm, prompt=script_template, verbose=True, output_key='script', memory=script_memory)  # Fixed this line

# Wikipedia API wrapper
def get_wikipedia_summary(query):
    wiki_wiki = wikipediaapi.Wikipedia('en')
    page = wiki_wiki.page(query)
    return page.summary[:1000]  # Limit summary length to 1000 characters

# Show results if there's a prompt
if prompt:
    # Generate the title
    title = title_chain.run(prompt)

    # Get Wikipedia research based on the prompt
    wiki_research = get_wikipedia_summary(prompt)

    # Generate the script based on the title and research
    script = script_chain.run(title=title, wikipedia_research=wiki_research)

    # Display the title and script
    st.write(f"**Generated Title:** {title}")
    st.write(f"**Generated Script:** {script}")

    # Expander for showing history
    with st.expander('Title History'):
        st.info(title_memory.buffer)

    with st.expander('Script History'):
        st.info(script_memory.buffer)

    with st.expander('Wikipedia Research'):
        st.info(wiki_research)


2024-09-06 07:00:39.118 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-06 07:00:39.119 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-06 07:00:39.123 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-06 07:00:39.125 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-06 07:00:39.127 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-06 07:00:39.129 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-06 07:00:39.130 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-06 07:00:39.131 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [25]:
pip install --upgrade langchain streamlit wikipedia-api
